In [1]:
#주피터 노트북에서 해당 모듈을 설치할때는 pip 명령어를 사용 ex) !pip install **** or pip install ***
!pip install config 

In [3]:
# 기본 셋팅! 사용할 import를 전부 설정
import os
import sys
import urllib.request
import datetime
import pandas as pd
import time
import json
from config import *

import matplotlib.pyplot as plt
import matplotlib

In [ ]:
# http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList

In [4]:
# [Code 1]

# import requests

# url = 'http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList'
# params ={'serviceKey' : '서비스키', 'YM' : '201201', 'NAT_CD' : '112', 'ED_CD' : 'D' }

# response = requests.get(url, params=params)
# print(response.content)

def getRequestUrl(url):
    
    req = urllib.request.Request(url)  # url 불러오기
        
    try: # 실행
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:  
            print("[%s] Url Request Success" %datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e: # 에러 예외처리
        print(e)
        print("[%s] Error for URL : % s" %(datetime.datetime.now(), url))
        return None

In [5]:
# [Code 2]

def getTourismStatusItem(yyyymm, nat_cd, ed_cd):
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"
    
    parameters = "?_type=json&serviceKey=" + serviceKey # 인증키
    parameters += "&YM" + yyyymm
    parameters += "&NAT_CD=" + nat_cd
    parameters += "&ED_CD=" + en_cd
    
    url = service_url + parameters
    
    responseDecode = getRequestUrl(url) # [Code 1]
    
    if(responseDecode == None):
        return None
    else:
        print("1")
        return json.loads(responseDecode)

In [6]:
# [Code 3]
def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonREsult = []
    result = []
    for year in range(nStartYear, nEndYear+1):
        for month in range(1, 13):
            yyyymm = "{0}{1:0>2}".format(str(year), str(month))
            jsonData = getTourismStatusItem(yyyymm, nat_cd, ed_cd) # [Code 2]
            if (jsonData['response']['header']['resultMsg'] == 'OK'):
                # 데이터가 없는 마지막 항목인 경우 -------------------------------
                if jsonData['response']['body']['items'] == '':
                    dataEND = "{0}{1:0>2}".format(str(year), star(month-1))
                    print("데이터 없음.... \n 제공되는 통계 데이터는 %s년 %s월까지 입니다." %(str(year), star(month-1)))
                    break
                # jsonData를 출력하여 확인 ...........................
                print(json.dumps(jsonData, indent = 4, sort_keys = True, ensure_ascii = False))
                dataEND = yyyymm
                natName = jsonData['response']['body']['items']['num']
                natName = naName.replace(' ', '')
                num = jsonData['response']['body']['items']['item']['num']
                ed = jsonData['response']['body']['items']['item']['ed']
                print('[ %s_%s : %s ]' %(natName, yyyymm, num))
                print('------------------------------------------------------------')
                jsonResult.append({'nat_name': natName, 'nat_cd': nat_cd, 'yyyymm': yyyymm, 'visit_cnt': num})
                result.append([natName, nat_cd, yyyymm, num])
                
    return jsonResult, result, natName, ed, dataEND

In [8]:
def main():
    
    # 중국: 112 / 일본: 130 / 미국: 275    
    print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
    nat_cd = input('국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) :')
    nStarYear = int(input('데이터를 몇 년부터 수집할까요? : '))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
    ed_cd = "E" #E : 방한외래관광객, D : 해외 출국